# Check if representative group knockoffs are working

In [1]:
# load packages for this tutorial
using Revise
using Knockoffs
using LinearAlgebra
using Random
using StatsBase
using Statistics
using ToeplitzMatrices
using Distributions
using Clustering
using ProgressMeter
using LowRankApprox
using Plots
using CSV, DataFrames
gr(fmt=:png);

# some helper functions to compute power and empirical FDR
function TP(correct_groups, signif_groups)
    return length(signif_groups ∩ correct_groups) / max(1, length(correct_groups))
end
function TP(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return TP(correct_groups, signif_groups)
end
function power(correct_snps, discovered_snps)
    return length(discovered_snps ∩ correct_snps) / length(correct_snps)
end
function FDR(correct_groups, signif_groups)
    FP = length(signif_groups) - length(signif_groups ∩ correct_groups) # number of false positives
    return FP / max(1, length(signif_groups))
end
function FDR(correct_groups, β̂, groups)
    signif_groups = get_signif_groups(β̂, groups)
    return FDR(correct_groups, signif_groups)
end
function get_signif_groups(β, groups)
    correct_groups = Int[]
    for i in findall(!iszero, β)
        g = groups[i]
        g ∈ correct_groups || push!(correct_groups, g)
    end
    return correct_groups
end

┌ Info: Precompiling Knockoffs [878bf26d-0c49-448a-9df5-b057c815d613]
└ @ Base loading.jl:1423
┌ Info: Precompiling ToeplitzMatrices [c751599d-da0a-543b-9d20-d0a503d91d24]
└ @ Base loading.jl:1423
┌ Info: Precompiling Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1423


get_signif_groups (generic function with 1 method)

## Simulate data with gnomdAD panel

In [2]:
datadir = "/Users/biona001/Benjamin_Folder/research/4th_project_PRS/group_knockoff_test_data"
covfile = CSV.read(joinpath(datadir, "CorG_2_127374341_128034347.txt"), DataFrame)
Σ = covfile |> Matrix{Float64}
Σ = 0.99Σ + 0.01I #ensure PSD

# test on smaller data
idx = 500 # 1241 # includes largest group with 192 members
Σ = Σ[1:idx, 1:idx];

# simulate data
m = 5
p = size(Σ, 1)
k = 10 # number of causal groups
n = 1000 # sample size

# simulate X
μ = zeros(p)
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups
nrep = 5
groups = id_partition_groups(X, force_contiguous=false)
unique_groups = unique(groups)
countmap(groups) |> values |> collect |> sort

136-element Vector{Int64}:
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  1
  ⋮
 14
 15
 16
 17
 18
 18
 21
 22
 25
 32
 44
 49

In [36]:
# group knockoffs
@time ko = modelX_gaussian_group_knockoffs(X, :maxent, groups, μ, Σ, m=m, verbose=true);

Maxent initial obj = -21697.062824271106
Iter 1 (PCA): obj = -17873.19705916164, δ = 2.7894170882622116, t1 = 0.2, t2 = 0.07
Iter 2 (PCA): obj = -14422.241803784958, δ = 0.9222596242085215, t1 = 0.37, t2 = 0.13
Iter 3 (PCA): obj = -12753.226804435964, δ = 0.5414552226809967, t1 = 0.58, t2 = 0.2
Iter 4 (PCA): obj = -11795.165658886217, δ = 0.4192851451235552, t1 = 0.86, t2 = 0.26
Iter 5 (PCA): obj = -11206.72302765663, δ = 0.48592870354058665, t1 = 1.07, t2 = 0.33
Iter 6 (PCA): obj = -10838.549047856957, δ = 0.6366331320900753, t1 = 1.33, t2 = 0.39
Iter 7 (PCA): obj = -10617.833462405928, δ = 0.6754313214808505, t1 = 1.57, t2 = 0.45
Iter 8 (PCA): obj = -10482.505674070737, δ = 0.27530848796806606, t1 = 1.79, t2 = 0.51
Iter 9 (PCA): obj = -10373.903657779418, δ = 0.17728527440017383, t1 = 1.95, t2 = 0.57
Iter 10 (PCA): obj = -10284.36124155696, δ = 0.2618276162815484, t1 = 2.1, t2 = 0.64
Iter 11 (CCD): obj = -10068.859996265239, δ = 0.5205009289960065, t1 = 2.38, t2 = 1.05, t3 = 0.0
Iter

In [37]:
# rep group knockoffs
@time rko = modelX_gaussian_rep_group_knockoffs(X, :maxent, μ, Σ, groups, verbose=true)

X̃ = rko.X̃
group_reps = rko.group_reps
Xr = X[:, group_reps]
Xc = X[:, setdiff(1:p, group_reps)]
X̃r = X̃[:, group_reps]
X̃c = X̃[:, setdiff(1:p, group_reps)];

140 representatives for 500 variables, 148 optimization variables
  0.416092 seconds (19.74 k allocations: 209.171 MiB)


In [38]:
count(!iszero, ko.S), count(!iszero, rko.S)

(8836, 230548)

Check if knockoffs generated from conditional independence assumption satisfy exchangability

In [39]:
# right column is the first 10 reps, left column is the group they belong to
[groups[group_reps[1:10]] group_reps[1:10]]

10×2 Matrix{Int64}:
  1   1
  2   2
  3   3
  4   4
  6   6
 90   8
  5  13
 17  38
  7  47
  8  49

In [40]:
groups[1:5]

5-element Vector{Int64}:
 1
 2
 3
 4
 6

In [8]:
cor(Xr)[1:5, 1:5]

5×5 Matrix{Float64}:
  1.0         -0.00637431  -0.122304    0.105602    0.0475231
 -0.00637431   1.0         -0.0649999   0.519294    0.0303531
 -0.122304    -0.0649999    1.0        -0.0991171  -0.0112321
  0.105602     0.519294    -0.0991171   1.0        -0.0572634
  0.0475231    0.0303531   -0.0112321  -0.0572634   1.0

In [9]:
cor(X̃r)[1:5, 1:5]

5×5 Matrix{Float64}:
  1.0         0.0171303   -0.118741     0.128614   -0.0119235
  0.0171303   1.0         -0.00397907   0.519748   -0.0560058
 -0.118741   -0.00397907   1.0         -0.0493806  -0.00172672
  0.128614    0.519748    -0.0493806    1.0        -0.0398285
 -0.0119235  -0.0560058   -0.00172672  -0.0398285   1.0

In [10]:
cor(Xc)[1:5, 1:5]

5×5 Matrix{Float64}:
  1.0        0.244665   0.253973  -0.432722  -0.434066
  0.244665   1.0        0.983266  -0.399884  -0.390473
  0.253973   0.983266   1.0       -0.398894  -0.387487
 -0.432722  -0.399884  -0.398894   1.0        0.979364
 -0.434066  -0.390473  -0.387487   0.979364   1.0

In [11]:
cor(X̃c)[1:5, 1:5]

5×5 Matrix{Float64}:
  1.0        0.381004   0.184924  -0.456294  -0.433225
  0.381004   1.0        0.659963  -0.505704  -0.381982
  0.184924   0.659963   1.0       -0.189855  -0.284863
 -0.456294  -0.505704  -0.189855   1.0        0.837525
 -0.433225  -0.381982  -0.284863   0.837525   1.0

**Conclusion**
Seems like $X_r$ and $\tilde{X}_r$ agrees fairly well while $X_c$ and $\tilde{X}_c$ agrees only somewhat. Lets try this in simulations

## One simulation

In [13]:
# simulate data
m = 5
p = size(Σ, 1)
k = 10 # number of causal groups
n = 500 # sample size
μ = zeros(p)

# simulate X
X = rand(MvNormal(μ, Σ), n)' |> Matrix
zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

# define groups
groups = id_partition_groups(X, force_contiguous=false)

# simulate βtrue
βtrue = zeros(p)
βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
shuffle!(βtrue)
causal_groups = get_signif_groups(βtrue, groups)

# simulate y
y = X * βtrue + randn(n)

# fully general me
@time me = modelX_gaussian_group_knockoffs(
    X, :maxent, groups, μ, Σ, 
    m = m,
    tol = 0.0001,    # convergence tolerance
    verbose=false, # whether to print informative intermediate results
)
me_ko_filter = fit_lasso(y, me)
me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
me_ssum = sum(abs.(me_ko_filter.ko.S))
@show me_power, me_fdr

# representative ME knockoffs
@time rme = modelX_gaussian_rep_group_knockoffs(
    X, :maxent, μ, Σ, groups, 
    m = m, 
);
rme_ko_filter = fit_lasso(y, rme)
discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
rme_power = round(TP(causal_groups, discovered_groups), digits=3)
rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
@show rme_power, rme_fdr

 14.216884 seconds (117.75 k allocations: 265.730 MiB, 0.09% gc time)
(me_power, me_fdr) = (0.286, 0.0)
  0.881965 seconds (19.85 k allocations: 390.918 MiB, 1.06% gc time)
(rme_power, rme_fdr) = (0.286, 0.0)


(0.286, 0.0)

### Interpolative decomposition, target FDR = 0.1, m=5

In [25]:
rme_fdr_hat = 0.0
rme_power_hat = 0.0
me_fdr_hat = 0.0
me_power_hat = 0.0

nsims = 10
for i in 1:nsims
    Random.seed!(i)
    
    # simulate data
    m = 5
    p = size(Σ, 1)
    k = 10 # number of causal groups
    n = 500 # sample size
    μ = zeros(p)

    # simulate X
    X = rand(MvNormal(μ, Σ), n)' |> Matrix
    zscore!(X, mean(X, dims=1), std(X, dims=1)); # standardize columns of X

    # define groups
    groups = id_partition_groups(X, force_contiguous=false)

    # simulate βtrue
    βtrue = zeros(p)
    βtrue[1:k] .= rand(-1:2:1, k) .* randn(k)
    shuffle!(βtrue)
    causal_groups = get_signif_groups(βtrue, groups)

    # simulate y
    y = X * βtrue + randn(n)

    # fully general me
    @time me = modelX_gaussian_group_knockoffs(
        X, :maxent, groups, μ, Σ, 
        m = m,
        tol = 0.0001,    # convergence tolerance
        verbose=false, # whether to print informative intermediate results
    )
    me_ko_filter = fit_lasso(y, me)
    me_power = round(TP(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_fdr = round(FDR(causal_groups, me_ko_filter.βs[3], groups), digits=3)
    me_ssum = sum(abs.(me_ko_filter.ko.S))
    @show me_power, me_fdr
    
    me_fdr_hat += me_fdr
    me_power_hat += me_power

    # representative ME knockoffs
    @time rme = modelX_gaussian_rep_group_knockoffs(
        X, :maxent, μ, Σ, groups, 
        m = m, 
    );
    rme_ko_filter = fit_lasso(y, rme)
    discovered_groups = groups[findall(!iszero, rme_ko_filter.βs[3])] |> unique
    rme_power = round(TP(causal_groups, discovered_groups), digits=3)
    rme_fdr = round(FDR(causal_groups, discovered_groups), digits=3)
    @show rme_power, rme_fdr
    
    rme_fdr_hat += rme_fdr
    rme_power_hat += rme_power
end
rme_fdr_hat /= nsims
rme_power_hat /= nsims
me_fdr_hat /= nsims
me_power_hat /= nsims
println("representative ME knockoff has avg power $rme_power_hat and FDR $rme_fdr_hat")
println("normal ME knockoff has avg power $me_power_hat and FDR $me_fdr_hat")

 13.573214 seconds (116.16 k allocations: 265.621 MiB, 0.07% gc time)
(me_power, me_fdr) = (0.25, 0.0)
  0.781274 seconds (19.64 k allocations: 379.382 MiB, 0.48% gc time)
(rme_power, rme_fdr) = (0.5, 0.0)
  9.439470 seconds (74.35 k allocations: 262.346 MiB, 0.04% gc time)
(me_power, me_fdr) = (0.222, 0.0)
  0.822142 seconds (19.83 k allocations: 390.208 MiB, 0.61% gc time)
(rme_power, rme_fdr) = (0.222, 0.0)
  9.172355 seconds (72.26 k allocations: 262.174 MiB)
(me_power, me_fdr) = (0.4, 0.2)
  0.925949 seconds (20.03 k allocations: 397.905 MiB, 23.96% gc time)
(rme_power, rme_fdr) = (0.5, 0.167)
  9.483176 seconds (70.27 k allocations: 261.993 MiB, 1.77% gc time)
(me_power, me_fdr) = (0.333, 0.0)
  0.752983 seconds (19.92 k allocations: 393.701 MiB, 0.40% gc time)
(rme_power, rme_fdr) = (0.444, 0.2)
 11.075415 seconds (90.86 k allocations: 263.741 MiB, 0.03% gc time)
(me_power, me_fdr) = (0.778, 0.0)
  0.830293 seconds (19.70 k allocations: 379.558 MiB, 0.40% gc time)
(rme_power, rm